<a href="https://colab.research.google.com/github/eyash24/Data-Science/blob/main/NLP_101_Fake_news_Classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake news classifier using LSTM
Following Krish Naik youtube playlist:
https://www.youtube.com/live/k3_qIfRogyY?si=QM2NAssSI_eOawd-

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"eyash24","key":"927bd2fd642245307c1a8effaf8ab2b2"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle competitions download -c fake-news

 95% 44.0M/46.5M [00:02<00:00, 29.4MB/s]
100% 46.5M/46.5M [00:02<00:00, 21.5MB/s]


In [5]:
! unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('train.csv')

In [16]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
# checking shape
df.shape

(20800, 5)

In [17]:
# checking for null information
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# dropping nan values
df = df.dropna()

In [20]:
df.shape

(18285, 5)

In [21]:
20800-18285

2515

In [9]:
# Independent Features
X = df.drop('label', axis=1)

In [10]:
y = df['label']

In [24]:
X.shape

(18285, 4)

In [25]:
y.shape

(18285,)

In [11]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [12]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [13]:
# vocabulary size
voc_size = 5000

## One Hot Representation

In [14]:
message = X.copy()

In [15]:
message['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
message

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [31]:
message.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [32]:
message

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [27]:
message['title'].isnull().sum()

0

In [33]:
message.head(20)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [34]:
for i in range(0, len(message)):
  print(message['title'][i])

Streaming output truncated to the last 5000 lines.
Mitch McConnell, Paul Ryan Meet with Donald Trump Promise ‘Bold Agenda’ - Breitbart
Susan Rice, Ex-Obama Adviser, Is Back in Political Cross Hairs Over Surveillance - The New York Times
The Fix Is In: NBC Affiliate Accidentally Posts Election Results A Week Early: Hillary Wins Presidency 42% to Trump’s 40%
Original unreleased transcript of Sonia Gandhi’s interview with Rajdeep Sardesai obtained
Indian Man Accused in Multimillion-Dollar Call-Center Swindle Is Held - The New York Times
7 Teenage Deaths, but No Answers for Aboriginal Canadians - The New York Times
Fashioning Cast-Iron Pans for Today’s Cooks - The New York Times
Summer Sweat: Hollywood Braces for Possible Worst Box Office in Decade
Trump Executive Orders Target Trade Abuses Amid NAFTA Intrigue - Breitbart
New York Times’s Moscow Bureau Was Targeted by Hackers - The New York Times
Barbra Streisand Says Sexism Cost Her Multiple Oscar Nominations
Comey Investigation a Red Her

In [35]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [38]:
# One Hot Representation
onehot_repr = [ one_hot(words, voc_size) for words in corpus ]
onehot_repr

[[70, 1364, 2145, 2026, 3041, 2604, 2377, 4063, 272, 3473],
 [3628, 4170, 3353, 3370, 4708, 2214, 2285],
 [4746, 642, 4651, 963],
 [2626, 2989, 3374, 4248, 4755, 4041],
 [1357, 4708, 4652, 1586, 1382, 4336, 4708, 3451, 2520, 1220],
 [2402,
  590,
  738,
  2364,
  3054,
  2859,
  4778,
  4991,
  169,
  1999,
  57,
  627,
  4564,
  1728,
  2285],
 [2679, 3047, 4510, 3514, 1262, 4751, 133, 2091, 461, 557, 4515],
 [265, 961, 1424, 4557, 3049, 4324, 2859, 1700, 461, 557, 4515],
 [2588, 1093, 2405, 4744, 3013, 2902, 1811, 2351, 2859, 2013],
 [800, 2447, 3889, 199, 842, 3800, 1234, 4716],
 [727, 3308, 2798, 2615, 4733, 4305, 849, 2865, 831, 1329, 3224],
 [4248, 1235, 3041, 2902, 2859, 3049],
 [2162, 1851, 1892, 1387, 508, 84, 4012, 2808, 273],
 [1522, 3075, 4532, 627, 2746, 1688, 2246, 461, 557, 4515],
 [2193, 3079, 3694, 2653, 214, 461, 557, 4515],
 [3784, 447, 3593, 1772, 3897, 1389, 560, 3328, 53, 4468],
 [4427, 4264, 4170],
 [2626, 4451, 207, 1087, 2859, 3636, 3178, 2285],
 [3937, 3393, 3

In [39]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [40]:
onehot_repr[1]

[3628, 4170, 3353, 3370, 4708, 2214, 2285]

## Embedding Representation

In [41]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4063  272 3473]
 [   0    0    0 ... 4708 2214 2285]
 [   0    0    0 ...  642 4651  963]
 ...
 [   0    0    0 ...  461  557 4515]
 [   0    0    0 ... 4528 2830 1754]
 [   0    0    0 ... 4400 3584 2019]]


In [42]:
# Model Creation
embedding_vector_features = 40

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
len(embedded_docs), y.shape

(18285, (18285,))

In [45]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [46]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [47]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [48]:
X_train.shape, X_test.shape

((12250, 20), (6035, 20))

In [49]:
# Training model
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 51ms/step - loss: 0.3270 - accuracy: 0.8584 - val_loss: 0.1947 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 8s 40ms/step - loss: 0.1390 - accuracy: 0.9451 - val_loss: 0.1939 - val_accuracy: 0.9130
Epoch 3/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0986 - accuracy: 0.9647 - val_loss: 0.2116 - val_accuracy: 0.9112
Epoch 4/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0698 - accuracy: 0.9750 - val_loss: 0.3001 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0485 - accuracy: 0.9860 - val_loss: 0.3589 - val_accuracy: 0.9157
Epoch 6/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0295 - accuracy: 0.9911 - val_loss: 0.3370 - val_accuracy: 0.9117
Epoch 7/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0163 - accuracy: 0.9953 - val_loss: 0.4250 - val_accuracy: 0.

In [50]:
# Adding dropout layer

from tensorflow.keras.layers import Dropout

# Re-creating the model
embedded_vector_feature = 50

model = Sequential()
model.add(Embedding(voc_size, embedded_vector_feature, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(150))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 50)            250000    
                                                                 
 dropout (Dropout)           (None, 20, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 150)               120600    
                                                                 
 dropout_1 (Dropout)         (None, 150)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 370751 (1.41 MB)
Trainable params: 370751 (1.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
# Training with new model
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 19s 78ms/step - loss: 0.3272 - accuracy: 0.8445 - val_loss: 0.1897 - val_accuracy: 0.9165
Epoch 2/10
192/192 [==============================] - 16s 84ms/step - loss: 0.1519 - accuracy: 0.9380 - val_loss: 0.1915 - val_accuracy: 0.9211
Epoch 3/10
192/192 [==============================] - 17s 87ms/step - loss: 0.1155 - accuracy: 0.9565 - val_loss: 0.2027 - val_accuracy: 0.9132
Epoch 4/10
192/192 [==============================] - 15s 76ms/step - loss: 0.0929 - accuracy: 0.9654 - val_loss: 0.2207 - val_accuracy: 0.9127
Epoch 5/10
192/192 [==============================] - 17s 89ms/step - loss: 0.0720 - accuracy: 0.9738 - val_loss: 0.2535 - val_accuracy: 0.9084
Epoch 6/10
192/192 [==============================] - 15s 76ms/step - loss: 0.0584 - accuracy: 0.9805 - val_loss: 0.3358 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 16s 82ms/step - loss: 0.0474 - accuracy: 0.9840 - val_loss: 0.3086 - val_accuracy:

## Performance Metrics and Accuracy

In [55]:
y_pred = model.predict(X_test)

189/189 [==============================] - 3s 14ms/step


In [56]:
y_pred = np.where(y_pred>0.5, 1, 0)
# use AUC ROC Curve to find threshold

In [58]:
from sklearn.metrics import confusion_matrix

In [59]:
confusion_matrix(y_test, y_pred)

array([[3154,  265],
       [ 310, 2306]])

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9047224523612262

In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      3419
           1       0.90      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

